In [1]:
# Mohd Uvas 18BCS079
# B.Tech(Computer Engineering 2018-2022 )
# Semester 8

#Problem:-
#Implement the Byte pair Encoding Algorithm of tokenization

# Getting the rules using training corpus

In [2]:
#taking file as input and returning vocabulary 
def get_vocab(text):

    text = text.replace('\n','')
    text=text.split()
    print(text)
    token_count=dict()
    
    for x in text:
        if x[-1]==',' or x[-1]=='.':
            x=x[:-1]

        x+='_'
        
        x_n=''
        for ch in x:
            x_n=x_n+ch+' '
        x=x_n[:-1]
        
        if x in token_count:
            token_count[x]+=1
        else:
            token_count[x]=1
            
    return token_count

In [3]:
#taking vocab as input and returning list of pair with thier frequency 
def get_pair_stats(vocab):
    pairs=dict()
    for word,frequency in vocab.items():
        symbols=word.split()
        
        for i in range(len(symbols)-1):
            pair=(symbols[i],symbols[i+1])
            current_frequency = pairs.get(pair,0)
            pairs[pair]=current_frequency+frequency
            
    return pairs

In [4]:
#taking pair to be merged and vocabulary as input and returning merged pair vocabulary
def merge_vocab(best_pair,vocab_in):
    vocab_out=dict()
    pattern = ' '.join(best_pair)
    replacement = ''.join(best_pair)
    for vb in vocab_in:
        vb_temp=vb
        if pattern in vb:
            vb=vb.replace(pattern,replacement)
            
        vocab_out[vb]=vocab_in[vb_temp]
        
    return vocab_out
            

In [5]:
#program for generating rules for merging 

file = open('train_data.txt')
text = file.read()
vocab = get_vocab(text)

print(vocab)

bpe_codes = dict()
num_merges = 20 
for i in range(num_merges):
    print('\niteration', i)
    pair_stats = get_pair_stats(vocab)
    if not pair_stats:
        break

    best_pair = max(pair_stats, key=pair_stats.get)
    bpe_codes[best_pair] = i

    print('vocabulary: ', vocab)
    print('best pair:', best_pair)
    vocab = merge_vocab(best_pair, vocab)

print('\nfinal vocabulary: ', vocab,len(vocab))
print('\nbyte pair encoding: ', bpe_codes)


['old', 'faster', 'older', 'old', 'older', 'finest.', 'faster', 'old', 'older', 'lowest', 'finest.', 'old', 'finest.', 'old', 'faster', 'faster', 'faster', 'faster', 'finest.', 'old', 'lowest', 'finest.', 'finest.', 'faster', 'lowest', 'old', 'finest.', 'finest.', 'finest.', 'lowest.', 'faster']
{'o l d _': 7, 'f a s t e r _': 8, 'o l d e r _': 3, 'f i n e s t _': 9, 'l o w e s t _': 4}

iteration 0
vocabulary:  {'o l d _': 7, 'f a s t e r _': 8, 'o l d e r _': 3, 'f i n e s t _': 9, 'l o w e s t _': 4}
best pair: ('s', 't')

iteration 1
vocabulary:  {'o l d _': 7, 'f a st e r _': 8, 'o l d e r _': 3, 'f i n e st _': 9, 'l o w e st _': 4}
best pair: ('e', 'st')

iteration 2
vocabulary:  {'o l d _': 7, 'f a st e r _': 8, 'o l d e r _': 3, 'f i n est _': 9, 'l o w est _': 4}
best pair: ('est', '_')

iteration 3
vocabulary:  {'o l d _': 7, 'f a st e r _': 8, 'o l d e r _': 3, 'f i n est_': 9, 'l o w est_': 4}
best pair: ('e', 'r')

iteration 4
vocabulary:  {'o l d _': 7, 'f a st er _': 8,

# Applying them on testing corpus for tokenization

In [6]:
#getting possible pairs for a list of character or string
def get_pairs(word):
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char

    return pairs


In [7]:
#creating new word after merging the pair into string
def create_new_word(word, pair_to_merge):
    first, second = pair_to_merge
    new_word = []
    i = 0
    while i < len(word):
        try:
            j = word.index(first, i)
            new_word.extend(word[i:j])
            i = j
        except ValueError:
            new_word.extend(word[i:])
            break

        if i < len(word) - 1 and word[i + 1] == second:
            new_word.append(first + second)
            i += 2
        else:
            new_word.append(first)
            i += 1

    return new_word


In [8]:
#combining the characters or string till no pair matching in byte pair
def encode(original_word, bpe_codes):
    if len(original_word) == 1:
        return original_word

    word = list(original_word)
    word.append('_')
    
    while True:
        pairs = get_pairs(word)
        bpe_codes_pairs = [(pair, bpe_codes[pair]) for pair in pairs if pair in bpe_codes]
        if not bpe_codes_pairs:
            break
            
        min_ptm=9999
        for pair in bpe_codes_pairs:
            if pair[1]<min_ptm:
                min_ptm=pair[1]
                pair_to_merge=pair[0]
                
        word = create_new_word(word, pair_to_merge)

        
    return word

In [9]:
#taking read file as input returning vocab without space 
def get_vocab_test(text_1):
    
    text_1 = text_1.replace('\n','')
    text_1=text_1.split()
    
    token_count_test=dict()
    
    for x in text_1:
        if x[-1]==',' or x[-1]=='.':
            x=x[:-1]
            x+='_'

        else:
            x+='_'
        
        if x in token_count_test:
            token_count_test[x]+=1
        else:
            token_count_test[x]=1
            
    return token_count_test

In [10]:
#opening test data file and tokenizing word
file_test = open('test_data.txt')
text_test = file_test.read()
vocab_test = get_vocab_test(text_test)

print("old_vocab\n",vocab_test,len(vocab_test))

static_vocab_test=[]
for st in vocab_test:
    static_vocab_test.append(st)
    
for oe_word in static_vocab_test:
    oe_word_temp=oe_word
    oe_word=oe_word[:-1]
    
    if len(oe_word)>2:
        oe_word_list=encode(oe_word, bpe_codes)
        if (len(oe_word_list)==2) and (len(oe_word_list[-1]) != 1):
            
            if oe_word_list[0] in vocab_test:
                vocab_test[oe_word_list[0]]+=vocab_test[oe_word_temp]
            else:
                vocab_test[oe_word_list[0]]=vocab_test[oe_word_temp]
            
            if oe_word_list[1] in vocab_test:
                vocab_test[oe_word_list[1]]+=vocab_test[oe_word_temp]
            else:
                vocab_test[oe_word_list[1]]=vocab_test[oe_word_temp]
            
            del vocab_test[oe_word_temp]

print("\n\n\nnew_vocab\n",vocab_test,len(vocab_test))

old_vocab
 {'oldest_': 7, 'finer_': 5, 'lower_': 5, 'lowest_': 5, 'fastest_': 5, 'finold_': 1, 'faster_': 5, 'finest_': 5, 'older_': 6} 9



new_vocab
 {'lowest_': 5, 'faster_': 5, 'finest_': 5, 'older_': 6, 'old': 7, 'est_': 12, 'fin': 6, 'er_': 10, 'low': 5, 'fast': 5, 'old_': 1} 11
